In [1]:
import IPython

load and display the sample audio file. We are going to use it to demo how the deepspeech2 model works.

In [2]:
audio_path = "./audio/eric.wav"
IPython.display.Audio(audio_path)

In [3]:
sample_transcript = "Hi I am Eric One of the available high quality text to speech voices select download now to install my voice"

In [4]:
from bigdl.util.common import *
import matplotlib.pyplot as plt

In [5]:
init_engine()

initialize the model hyper-parameters

In [6]:
sample_rate = 16000
window_size = 400
window_stride = 160
utt_length = 30 * (sample_rate / window_stride)
maxT_scrptLen = 500
num_filter = 13

In [7]:
import soundfile as sf
data, samplerate = sf.read(audio_path)


In [8]:
from pyspark.sql import SparkSession
conf = create_spark_conf()
spark = SparkSession.builder.appName("DeepSpeech Inference").getOrCreate()
spark_df = spark.createDataFrame(sc.parallelize([[data.tolist(), sample_transcript]]), schema=["data_vector","transcript"])

In [9]:
spark_df.show()

+--------------------+--------------------+
|         data_vector|          transcript|
+--------------------+--------------------+
|[WrappedArray(0.0...|Hi I am Eric One ...|
+--------------------+--------------------+



In [10]:
from py4j.java_gateway import java_import

In [11]:
alphabet = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "
model_path = "./ds2.model"

In [12]:
transformers = sc._jvm.com.intel.analytics.zoo.pipeline.deepspeech2.pipeline.acoustic

In [13]:
segmenter = transformers.TimeSegmenter().setSegmentSize(sample_rate * maxT_scrptLen) \
                                        .setInputCol("samples") \
                                        .setOutputCol("segments") 
        
windower = transformers.Windower().setInputCol("segments") \
                                  .setOutputCol("window") \
                                  .setOriginalSizeCol("originalSizeCol") \
                                  .setWindowShift(window_stride) \
                                  .setWindowSize(window_size) 
                
dft_specgram = transformers.DFTSpecgram().setInputCol("window") \
                                         .setOutputCol("specgram") \
                                         .setWindowSize(window_size)
        
mel_bank = transformers.MelFrequencyFilterBank().setInputCol("specgram") \
                                                .setOutputCol("mel") \
                                                .setWindowSize(window_size) \
                                                .setNumFilters(num_filter) \
                                                .setUttLength(utt_length) 
                
transpose_flip = transformers.TransposeFlip().setInputCol("mel") \
                                             .setOutputCol("features") \
                                             .setNumFilters(num_filter)
        
model_transformer = transformers.DeepSpeech2ModelTransformer(model_path).setInputCol("features") \
                                                                        .setOutputCol("prob") \
                                                                        .setNumFilters(num_filter)
decoder = transformers.VocabDecoder(model_path).setInputCol("prob") \
                                               .setOutputCol("output") \
                                               .setOriginalSizeCol("originalSizeCol") \
                                               .setAlphabet(alphabet) \
                                               .setUttLength(utt_length) \
                                               .setWindowSize(window_size)



In [14]:
stages = [segmenter, windower, dft_specgram, mel_bank, transpose_flip, model_transformer, decoder]
pipeline = Pipeline().setStages(stages)

NameError: name 'Pipeline' is not defined

In [ ]:
model = pipeline.fit(spark_df)

In [18]:
result = model.transform(spark_df).select("output", "target").rdd.cache()

NotImplementedError: 

In [19]:
result.collect()

NameError: name 'result' is not defined